# Mediapipe face mesh

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
# Import required libraries to run the naive baseline
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from collections import OrderedDict
import matplotlib.pyplot as plt
import pickle

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.models import mobilenet_v3_small

In [ ]:
# Test with the mediapipe face mesh pretrained model
# pip install mediapipe



# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Load your image
path = df_train.iloc[0]['filename']
image = cv2.imread(path)
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Process the image to find face landmarks
results = face_mesh.process(rgb_image)

# Create a mask for the skin area
mask = np.zeros_like(image)

# Define indices for the facial landmarks that typically represent the skin area
# These are example indices and may need refinement for your specific case
skin_landmark_indices = list(range(0, 468))  # Indices for all landmarks

if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:
        points = []
        for idx in skin_landmark_indices:
            x = int(face_landmarks.landmark[idx].x * image.shape[1])
            y = int(face_landmarks.landmark[idx].y * image.shape[0])
            points.append([x, y])
        
        points = np.array(points)
        
        # Create a convex hull around the skin landmarks to approximate the skin area
        hull = cv2.convexHull(points)
        
        # Draw the convex hull on the mask
        cv2.fillConvexPoly(mask, hull, (255, 255, 255))

# Apply the mask to the original image
skin_area = cv2.bitwise_and(image, mask)

# Display the mask and the resulting skin area
cv2.imshow('Mask', mask)
cv2.imshow('Skin Area', skin_area)
cv2.waitKey(0)
cv2.destroyAllWindows()

